# DFFRAM Compiler

Standard Cell Library based Memory Compiler using DFF/Latch cells.


## Warning!

This notebook should still work, but is out of date and does not reflect the
latest DFFRAM release. Use at your own risk!

### Overview
DFFRAM is based around two Python modules: the DFFRAM flow and the placeram utility.

placeram is a custom placer using OpenROAD's Python interface. It places DFFRAM RAM/register file designs in a predetermined structure to avoid a lengthy and inefficient manual placement process for RAM. 

dffram.py is a relatively self-contained flow that uses placeram and OpenLane to place, route and harden RAM. 

### Supported Sizes 

> 1RW1R variants are temporarily disabled due to a bug.

RAM: 
- 32 words with byte write enable (1RW and 1RW1R).
- 128 words with byte write enable (1RW and 1RW1R).
- 256 words with byte write enable (1RW and 1RW1R).
- 512 words with byte write enable (1RW and 1RW1R).
- 1024 words with byte write enable (1RW and 1RW1R).
- 2048 words with byte write enable (1RW and 1RW1R).


RF: 

- 32 x 32-bit words (2R1W)



---



### Installing Dependencies

In [ ]:
openlane_version = ''
open_pdks_version = '' 

import os
import pathlib

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version}
!python -m pip install gdstk gdstk
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}

### Cloning DFFRAM and Installing the required dependencies:  🔄

(We're also setting some environment variables that make OpenLane and DFFRAM play nicer with conda.)

In [ ]:
!apt-get update && apt-get install -yq tcllib
%env TCLLIBPATH=/usr/share/tcltk

%cd /content
!rm -rf ./dffram
!mkdir -p ./dffram
!curl -L https://github.com/AUCOHL/DFFRAM/tarball/a640412d65e2518858a6433dc53839a67b23572b | tar -xzC ./dffram --strip-components=1
!python3 -m pip install -r /content/dffram/requirements.txt

%cd /content/dffram
%env NO_CHECK_INSTALL=1



---



### For a full list of different options 🤔  

In [ ]:
!./dffram.py --help

### sky130A 

#### 32x32 with one read-write port

In [ ]:
%env PDK=sky130A
!./dffram.py --using-local-openlane /content/conda-env/share/openlane -s 32x32

#### 32x32 with two read ports and one write port (a register file) 

In [ ]:
%env PDK=sky130A
!./dffram.py --using-local-openlane /content/conda-env/share/openlane -s 32x32 -b ::rf -v 2R1W

### gf180mcuC

#### 32x32 with one read-write port

In [ ]:
%env PDK=gf180mcuC
!./dffram.py --using-local-openlane /content/conda-env/share/openlane -s 32x32 -b gf180mcuC:gf180mcu_fd_sc_mcu7t5v0:ram

#### 32x32 with two read ports and one write port (a register file)

In [ ]:
%env PDK=gf180mcuC
!./dffram.py --using-local-openlane /content/conda-env/share/openlane -s 32x32 -b gf180mcuC:gf180mcu_fd_sc_mcu7t5v0:rf -v 2R1W

### Aaand visualize it! 🎉

In [ ]:
import pathlib
import gdstk
from IPython.display import SVG

gds = sorted(
    pathlib.Path("/content/dffram/build/32x32_DEFAULT").glob("**/gds/RAM32.gds")
)[0]
library = gdstk.read_gds(gds)
top_cells = library.top_level()
top_cells[0].write_svg("RAM32.svg")
SVG("RAM32.svg")